In [5]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/lib'))
sys.path.append(str('/home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API
import torch.nn.functional as F
from weight_initializers import init_net


def pruning_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()   
      

    for name, param in network.module.state_dict().items():
        if name=='arch_parameters':###whether abs on arch parameter
            print(param)
            param.abs_()
        else:
            param.abs_()
    arch_pruned=deepcopy(network.module.arch_parameters.data)
    
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))

        base_targets=torch.tensor(base_targets).to(torch.float32)
        arch_targets=torch.tensor(arch_targets).to(torch.float32)
        # print(base_inputs)
        # print(base_targets)
        print(args.labelrandom)
        if args.labelrandom=='random':
            base_targets=torch.randn_like(base_targets).cuda()
            arch_targets=torch.randn_like(arch_targets).cuda()
        elif args.labelrandom=='allone':
            base_targets=torch.ones_like(base_targets).cuda()
            arch_targets=torch.ones_like(arch_targets).cuda()
            # arch_inputs=torch.ones_like(arch_inputs).cuda()            
        elif args.labelrandom=='data':
            base_targets = base_targets.cuda(non_blocking=True)
            arch_targets = arch_targets.cuda(non_blocking=True)
        else:
            NotImplementedError
        # measure data loading time

        # update the architecture-weight
        a_optimizer.zero_grad()
        
        input_dim = list(arch_inputs[0,:].shape)
        # inputs = torch.ones([1] + input_dim).float().cuda(non_blocking=True)
        if args.datarandom=='random':
            inputs=torch.randn_like(base_inputs).cuda()
            # arch_inputs=torch.randn_like(arch_inputs).cuda()
        elif args.datarandom=='allone':
            inputs=torch.ones_like(base_inputs).cuda()
            # arch_inputs=torch.ones_like(arch_inputs).cuda()            
        elif args.datarandom=='data':
            inputs=base_inputs
        else:
            NotImplementedError
            
        # print(inputs)
        _, logits = network.forward(inputs)        
        
        arch_loss = torch.sum(logits)
        arch_loss.backward()
        #a_optimizer.step()
        #arch_synflow=abs(F.softmax(network.module.arch_parameters.data)*network.module.arch_parameters.grad.data)
        #arch_synflow=abs(network.module.arch_parameters.grad.data)        
        # arch_synflow=abs(network.module.arch_parameters.data*network.module.arch_parameters.grad.data)        
        # print(network.module.arch_parameters.data)
        # print(network.module.arch_parameters.grad.data)   
        #####################################################################################
        #break
        num_batchs=1
        if step%num_batchs==0:         
            synflow=np.zeros(5)            
            availabe_edge=[]
            for edge in range(6):
                num_of_oper=0
                for i in range(5):
                    if arch_pruned[edge,i]> -float('Inf'):
                        num_of_oper +=1
                if num_of_oper>1:
                    availabe_edge.append(edge)
            edge_ind= np.random.randint(len(availabe_edge))   
            edge=availabe_edge[edge_ind]
            
            num_of_oper=0
            for i in range(5):
                if arch_pruned[edge,i]> -float('Inf'):
                    # snip[i]=abs(network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i])
                    synflow[i]= network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i]
                    num_of_oper +=1
                else:
                    synflow[i]= float('Inf')
        else:
            synflow_new=np.zeros(5)
            for i in range(5):
                if arch_pruned[edge,i]> -float('Inf'):
                    synflow_new[i]= network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i]
                    # snip_new[i]=abs(network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i])

            synflow = synflow + synflow_new    
                    
        if (step+1)%num_batchs==0:      
            if num_of_oper>1:
                ind=np.argmin(synflow)
                arch_pruned[edge,ind]=-float('Inf')
            logger.log('{:}'.format(api.query_by_arch(network.module.genotype_prune(arch_pruned))))  
            total_ope=0
            for edge in range(6):
                for i in range(5):
                    if arch_pruned[edge,i]> -float('Inf'):
                        total_ope +=1
            print(arch_pruned)
            print('---------------------------------------------')                                                
            if total_ope==6:
                break
    
    print('kaishi-----------------------------------')  
            
    return arch_pruned


parser = argparse.ArgumentParser("DARTS First Order")
parser.add_argument('--data_path',          type=str,   default= '/home/cwang4/Data/Datasets', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-2, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/NAS-Bench-201-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=50,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=327, help='manual seed')
parser.add_argument('--datarandom',            type=str,   default= 'data', choices=['data', 'random', 'allone'], help='.')
parser.add_argument('--labelrandom',            type=str,   default= 'random', choices=['data', 'random', 'allone'], help='.')

args = parser.parse_args([])

In [6]:
args.save_dir= './00FreeDARTS_SynFlow_seed_'+str(args.rand_seed)+args.save_dir

xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.cuda.set_device(0)
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config_path = '/home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/configs/nas-benchmark/algos/DARTS.config'
config = load_config(config_path, {'class_num': class_num, 'xshape': xshape}, logger)
# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'DARTS-V1', 'C': xargs.channel, 'N': xargs.num_cells,
                              'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                              'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)
# logger.log('search-model :\n{:}'.format(search_model))


w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)


logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()


  # start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
epoch=0
w_scheduler.update(0, 0.0)

arch_pruned = pruning_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, xargs.print_freq, logger)
search_time.update(time.time() - start_time)
logger.log('{:}'.format(api.query_by_arch(network.module.genotype_prune(arch_pruned))))   
logger.log('Pruning cost {:.1f} s.'.format(search_time.sum))
logger.log('Pruning finished')
logger.log('-----------------------------------------------------------------')
# if api is not None: logger.log('{:}'.format(api.query_by_arch(network.module.genotype_prune(arch_pruned)))) 
logger.close()

Main Function with logger : Logger(dir=00FreeDARTS_SynFlow_seed_327/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.03
arch_nas_dataset : /home/cwang4/Data/Projects/AngleNAS-miao/NAS_Bench201/NAS-Bench-201-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
data_path        : /home/cwang4/Data/Datasets
datarandom       : data
dataset          : cifar10
labelrandom      : random
max_nodes        : 4
num_cells        : 5
print_freq       : 50
rand_seed        : 327
save_dir         : ./00FreeDARTS_SynFlow_seed_327/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
workers          : 2
Python  Version  : 3.8.16 (default, Mar  2 2023, 03:21:46)  [GCC 11.2.0]
Pillow  Version  : 9.4.0
PyTorch Version  : 1.12.1+cu113
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 1
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloa

/tmp/ipykernel_15299/761240300.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  base_targets=torch.tensor(base_targets).to(torch.float32)
/tmp/ipykernel_15299/761240300.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  arch_targets=torch.tensor(arch_targets).to(torch.float32)


random
|skip_connect~0|+|nor_conv_3x3~0|skip_connect~1|+|skip_connect~0|skip_connect~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=10032
cifar10-valid  FLOP= 43.17 M, Params=0.316 MB, latency=12.08 ms.
cifar10-valid  train : [loss = 0.645, top1 = 77.35%], valid : [loss = 0.748, top1 = 74.42%]
cifar10        FLOP= 43.17 M, Params=0.316 MB, latency=11.82 ms.
cifar10        train : [loss = 0.522, top1 = 82.05%], test  : [loss = 0.573, top1 = 80.53%]
cifar100       FLOP= 43.18 M, Params=0.322 MB, latency=11.76 ms.
cifar100       train : [loss = 2.000, top1 = 46.39%], valid : [loss = 2.102, top1 = 44.17%], test : [loss = 2.100, top1 = 44.35%]
ImageNet16-120 FLOP= 10.80 M, Params=0.323 MB, latency=9.51 ms.
ImageNet16-120 train : [loss = 3.169, top1 = 23.59%], valid : [loss = 3.123, top1 = 24.93%], test : [loss = 3.175, top1 = 23.83%]
tensor([[5.1758e-04, 1.7865e-03, 2.4616e-04, 6.3276e-04, 6.3033e-04],
        [6.3756e-04, 4.